In [ ]:
# import settings and functions
%run ./../../imports.ipynb


## What mesh?

Copy your choice to the next cell

for SquareTop:
```
analytical_solution_tag = "-ana_square_top"
generate_config = generateConfig_squareTop
generate_mesh = generateMesh_squareTop
```

for SquareSinCos:
```
analytical_solution_tag = "-ana_square_sincos"
generate_config = generateConfig_squareSinCos
generate_mesh = generateMesh_squareSinCos
```

In [2]:
# Change according to instruction above
# analytical_solution_tag = "-ana_mexi_hat"
# generate_config = generateConfig_squareMexiHat
# generate_mesh = generateMesh_squareMexiHat

# analytical_solution_tag = "-ana_square_top"
# generate_config = generateConfig_squareTop
# generate_mesh = generateMesh_squareTop

analytical_solution_tag = "-ana_square_sincos"
generate_config = generateConfig_squareSinCos
generate_mesh = generateMesh_squareSinCos

## Analysis setup

In [3]:
# which executable?

exe = data_driven_diffusion_snes
sumanalys = "sumanalys.csv"
ana_name = "ana_square_sincos_DD_order"

ana_compare_exe = [data_driven_diffusion_snes, classic_diffusion]
ana_compare_name = ["ana_square_sincos_mixed", "ana_square_sincos_classic"]
# ana_compare_name = ["ana_square_sincos_mixed"]
ana_compare_sum = ["sumanalys.csv", "FEM_errors.csv"]

# Convergence analysis parameters
order_list = [1, 2, 3] # approximation order p
elem_size_list = [0.1, 0.05, 0.02, 0.01] # element size h
# order_list = [1, 2, 3] # approximation order p
# elem_size_list = [0.5, 0.2, 0.1] # element size h
params.triangle_mesh = True
params.nproc = 1 # number of processors
jumps = ""
if params.nproc == 1:
    jumps = "-get_jumps"
# jumps = "-get_jumps"

use_line = "-use_line"
# use_line = ""

run_test = True
run_analysis = True
run_refinement_analysis = True
run_refinement_mesh_analysis = True
run_refinement_hp_analysis = True

run_test = False
run_analysis = False
run_refinement_analysis = False
run_refinement_mesh_analysis = False
run_refinement_hp_analysis = False

naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorEstimator",
          "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter", "errorIndicatorGrad", "errorIndicatorDiv", "jumpL2", "jumpHdiv", "eleNum"]
# naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorEstimator",
#           "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter"]

error_name_list = ["L2norm", "H1seminorm", "fluxErr"]
error_label_list = [(r'Global error $L^2$-norm'),
               (r'Global error $H^1$-seminorm'), (r'Global Flux error')]

In [4]:
params.conductivity = 1.0 # linear conductivity
params.element_size = 0.05 # element size in the regular mesh
params.order = 2 # approximation order for displacements

# params.triangle_mesh = False # use triangular mesh

# Pre-processing parameters
params.mesh_file = "square_sincos"
params.length_x = 1
params.length_y = 1
params.length_z = 0
params.show_mesh = True


# solution parameters
params.log_file = "log" # log file name 


## Run test

In [ ]:
# start display for showing results
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()

In [6]:
# Testing mesh generation
if run_test:
    params.show_mesh = True
    generate_config(params)
    generate_mesh(params)

In [7]:
# Testing running analysis
params.order = 2
if run_test:
    !rm out*
    params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
    !{mofem_part} -my_file {params.mesh_file + ".h5m"} -nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1
    !mpirun -np {params.nproc} {exe} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag} {jumps} {use_line} 
    # -print_integ

    # !convert.py out*


In [8]:
if run_test:    
    out_to_vtk = !ls -c1 out_integ_pts_*.h5m
    last_file=out_to_vtk[0]
    !convert.py {last_file} 
    out_to_vtk = !ls -c1 out_resu*.h5m
    last_file=out_to_vtk[0]
    !convert.py {last_file} 
    out_to_vtk = !ls -c1 out_ite*.h5m
    last_file=out_to_vtk[0]
    !convert.py {last_file} 
    out_to_vtk = !ls -c1 out_error*.h5m
    last_file=out_to_vtk[0]
    !convert.py {last_file} 
    !rm out*h5m

In [9]:
if run_test:
    params.show_file = "out_resu"
    params.show_field = "T"
    params.warp_field_scalar = "T"
    params.show_edges = False
    params.p_cmap = color_temperature
    params.clim = (0, 1)
    params.warp_factor = 0.8
    params.camera_position =  [
            (-1.3, -1.3, 1.1),
            (0.35, 0.35, 0),
            (0.25, 0.25, 1),
        ]
    # params.p_cmap = "jet"
    params.p_save = "c4_T_ord_"+str(params.order)+".pdf"
    show_results(params)


    params.clim = None
    params.camera_position = "xy"
    params.warp_field_scalar = ""


In [10]:
if run_test:
    params.field_part = 10
    params.show_file = "out_resu"
    params.show_field = "T"
    params.show_edges = True
    params.clim = [0, 10]
    # params.clim = None
    params.p_cmap = color_gradient
    params.p_save = "c4_grad_T.pdf"
    show_results(params)
    params.field_part = -1

In [11]:
if run_test:
    params.field_part = -1
    params.show_file = "out_resu"
    params.show_field = "Q"
    params.show_edges = True
    params.p_cmap = color_gradient
    params.p_save = "c4_Q.pdf"
    show_results(params)

## Comparison between standard and mixed

In [12]:
if run_analysis:    
    for i in range(len(ana_compare_name)):
        !rm {ana_compare_sum[i]}
        !rm ./out_*
        for elem_size in elem_size_list:
            params.element_size = elem_size
            params.show_mesh = False
            generate_mesh(params)
            params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
            !{mofem_part} -my_file {params.mesh_file + ".h5m"} -my_nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1
            for order in order_list:
                params.order = order
                !mpirun -np {params.nproc} {ana_compare_exe[i]} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag} {jumps} {use_line}
        !mv {ana_compare_sum[i]} {ana_compare_name[i]}.csv
    


### Load analysis

In [ ]:
print(naming)

In [14]:
error_name_list = []
error_label_list = []

error_name_list.append("L2norm")
error_label_list.append(r'Global error $L^2$-norm')
error_name_list.append("H1seminorm")
error_label_list.append(r'Global error $H^1$-seminorm')
error_name_list.append("fluxErr")
error_label_list.append(r'Global Flux error')



In [15]:
filename_prefix = "c4_DD_sincos_"

DD_ana = Analysis(ana_compare_name[0], naming, order_list, error_name_list, error_label_list, filename_prefix, elem_size_list,  marker='x', linestyle='--', plot_gradients=True, label="DD")
classic_ana = Analysis(ana_compare_name[1], naming, order_list, error_name_list, error_label_list, filename_prefix, elem_size_list,  marker='+', linestyle=':', plot_gradients=False, label="Standard")

### Plot results

In [ ]:

DD_ana.plot_both_analyses_by_elem_size([])
DD_ana.plot_both_analyses_by_gaussnum([])

In [ ]:
DD_ana.plot_gradients = False
DD_ana.legend_fond_size = 12

DD_ana.plot_both_analyses_by_gaussnum([classic_ana])
DD_ana.plot_both_analyses_by_elem_size([classic_ana])
